In [1]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [4]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

hub = {
	'HF_MODEL_ID':'Qwen/Qwen2.5-VL-3B-Instruct',
	'SM_NUM_GPUS': json.dumps(1)
}

huggingface_model = HuggingFaceModel(
	transformers_version='4.49.0',
	pytorch_version='2.6.0',
	py_version='py312',
	env=hub,
	role=role, 
)

In [5]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
)

------------!

In [6]:
predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

[{'generated_text': "Hi, what can you help me with? I'm looking for a way to create a website that is both visually appealing and easy to navigate."}]

In [7]:
predictor.delete_model()
predictor.delete_endpoint()